In [1]:
import warnings
warnings.filterwarnings('ignore')

from astropy.table import Table
import numpy as np
import astropy.constants as ac
import astropy.units as au
from glob import glob
from spectral_cube import SpectralCube
import reproject 
import concurrent.futures
from astropy.io import fits
import os
from tqdm.auto import tqdm 
import gc 

from analysis_phangs_hst import dendro_cutouts, dendro_misc

In [2]:
# Replace with your file dirs here... 
# 
dir_hst = '../../../data/hst_cropped/'
hst01_file = '%s/ngc628_uvis_f275w_exp_drc_sci.fits' %dir_hst
hst02_file = '%s/ngc628_uvis_f336w_exp_drc_sci.fits' %dir_hst
hst03_file = '%s/ngc628_acs_f435w_exp_drc_sci.fits' %dir_hst
hst04_file = '%s/ngc628_uvis_f555w_exp_drc_sci.fits' %dir_hst
hst05_file = '%s/ngc628_acs_f814w_exp_drc_sci.fits' %dir_hst
hst06_file = '../../../data/hst_contsub/f555w_f814w/ngc628_halpha_raw.fits'
hst07_file = '../../../data/hst_contsub/f555w_f814w/ngc628_halpha_fit_anchored_intnegs.fits'

# 
musha_file = '../../../data/muse/maps/HA6562_FLUX.fits'
almaco_file = '../../../data/alma/ngc0628_12m+7m+tp_co21_broad_tpeak.fits'
almaco1_file = '../../../data/alma/ngc0628_12m+7m+tp_co21_broad_mom0.fits'
vlahi_file = '../../../data/vla/NGC_628_NA_MOM0_THINGS.fits'

# 
dir_hst = '../../../data/jwst_updated_v0p4p0/'
jwst01_file = '%s/ngc0628_nircam_lv3_f200w_i2d_align.fits' %dir_hst
jwst02_file = '%s/ngc0628_nircam_lv3_f300m_i2d_align.fits' %dir_hst
jwst03_file = '%s/ngc0628_nircam_lv3_f335m_i2d_align.fits' %dir_hst
jwst04_file = '%s/ngc0628_nircam_lv3_f360m_i2d_align.fits' %dir_hst

dir_hst = '../../../data/jwst_updated_v0p5_miri/'
jwst05_file = '%s/ngc0628_miri_f770w_anchored.fits' %dir_hst
jwst06_file = '%s/ngc0628_miri_f1000w_anchored.fits' %dir_hst
jwst07_file = '%s/ngc0628_miri_f1130w_anchored.fits' %dir_hst
jwst08_file = '%s/ngc0628_miri_f2100w_anchored.fits' %dir_hst


dir_hst = '../../../data/jwst_feast/'
jwst09_file = '%s/jw01783-o004_t008_nircam_clear-f115w_i2d_J2000.fits' %dir_hst
jwst10_file = '%s/jw01783-o004_t008_nircam_clear-f150w_i2d_J2000.fits' %dir_hst
jwst11_file = '%s/jw01783-o004_t008_nircam_clear-f187n_i2d_J2000.fits' %dir_hst
jwst12_file = '%s/jw01783-o004_t008_nircam_clear-f187n_ONLY_J2000.fits' %dir_hst
jwst13_file = '%s/jw01783-o004_t008_nircam_clear-f277w_i2d_J2000.fits' %dir_hst
jwst14_file = '%s/jw01783-o004_t008_nircam_clear-f444w_i2d_J2000.fits' %dir_hst
jwst15_file = '%s/jw01783-o004_t008_nircam_f405_ONLY_J2000.fits' %dir_hst
jwst16_file = '%s/jw01783-o004_t008_nircam_f405n-f444w_i2d_J2000.fits' %dir_hst

asto_file = '../../../data/astrosat/NGC0628_FUV_ave.fits'
spit_file = '../../../data/spitzer/NGC_0628_IRAC_I4_bms2014.fits'

# Loading catalouges
muscat_file = '../../../data/muse/catalouge/NGC0628_nebulae_mask_V2.fits' #Muse Neb catalouge

# 
dir_asscat = '../../../data/cluster_catalogs/ngc628-vselect/'
asscat1_file = '%s/ws16pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws16pc-idmask.fits' %dir_asscat #HST association catalogue
asscat2_file = '%s/ws32pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws32pc-idmask.fits' %dir_asscat #HST association catalogue
asscat3_file = '%s/ws64pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws64pc-idmask.fits' %dir_asscat #HST association catalogue

In [3]:
# # loading files
# # changing everything into log scale
hst01_hdu = fits.open(hst01_file)[0]
hst02_hdu = fits.open(hst02_file)[0]
hst03_hdu = fits.open(hst03_file)[0]
hst04_hdu = fits.open(hst04_file)[0]
hst05_hdu = fits.open(hst05_file)[0]
hst06_hdu = fits.open(hst06_file)[0]
hst07_hdu = fits.open(hst07_file)[0]

# 
musha_hdu = fits.open(musha_file)[1]
musha_hdu.data[np.isnan(musha_hdu.data)] = -100

almaco_hdu = fits.open(almaco_file)[0]
almaco1_hdu = fits.open(almaco1_file)[0]

vlahi_hdu = fits.open(vlahi_file)[0]
vlahi_hdu.data = np.squeeze(vlahi_hdu.data)
del vlahi_hdu.header['*3*']
del vlahi_hdu.header['*4*']

#
jwst01_hdu = fits.open(jwst01_file)[1]
jwst02_hdu = fits.open(jwst02_file)[1]
jwst03_hdu = fits.open(jwst03_file)[1]
jwst04_hdu = fits.open(jwst04_file)[1]
jwst05_hdu = fits.open(jwst05_file)[0]
jwst06_hdu = fits.open(jwst06_file)[0]
jwst07_hdu = fits.open(jwst07_file)[0]
jwst08_hdu = fits.open(jwst08_file)[0]

jwst01_hdu.data[jwst01_hdu.data == 0] = np.nan
jwst02_hdu.data[jwst02_hdu.data == 0] = np.nan
jwst03_hdu.data[jwst03_hdu.data == 0] = np.nan
jwst04_hdu.data[jwst04_hdu.data == 0] = np.nan
jwst05_hdu.data[jwst05_hdu.data == 0] = np.nan
jwst06_hdu.data[jwst06_hdu.data == 0] = np.nan
jwst07_hdu.data[jwst07_hdu.data == 0] = np.nan
jwst08_hdu.data[jwst08_hdu.data == 0] = np.nan

jwst09_hdu = fits.open(jwst09_file)[0]
jwst10_hdu = fits.open(jwst10_file)[0]
jwst11_hdu = fits.open(jwst11_file)[0]
jwst12_hdu = fits.open(jwst12_file)[0]
jwst13_hdu = fits.open(jwst13_file)[0]
jwst14_hdu = fits.open(jwst14_file)[0]
jwst15_hdu = fits.open(jwst15_file)[0]
jwst16_hdu = fits.open(jwst16_file)[0]

jwst09_hdu.data[jwst09_hdu.data == 0] = np.nan
jwst10_hdu.data[jwst10_hdu.data == 0] = np.nan
jwst11_hdu.data[jwst11_hdu.data == 0] = np.nan
jwst12_hdu.data[jwst12_hdu.data == 0] = np.nan
jwst13_hdu.data[jwst13_hdu.data == 0] = np.nan
jwst14_hdu.data[jwst14_hdu.data == 0] = np.nan
jwst15_hdu.data[jwst15_hdu.data == 0] = np.nan
jwst16_hdu.data[jwst16_hdu.data == 0] = np.nan

#
asto_hdu = fits.open(asto_file)[0]
spit_hdu = fits.open(spit_file)[0]

data, _ = reproject.reproject_interp(spit_hdu, musha_hdu.header)
spit_hdu = fits.PrimaryHDU(data, musha_hdu.header)

# loading catalouges
muscat_hdu = fits.open(muscat_file)[0]
asscat1_hdu = fits.open(asscat1_file)[0]
asscat2_hdu = fits.open(asscat2_file)[0]
asscat3_hdu = fits.open(asscat3_file)[0]

muscat_hdu.data = np.array(muscat_hdu.data, dtype=float)
muscat_hdu.data[muscat_hdu.data==-1] = np.nan

asscat1_hdu = fits.PrimaryHDU(reproject.reproject_interp(asscat1_hdu, hst01_hdu.header, return_footprint=False, order='nearest-neighbor', parallel=True), hst01_hdu.header)
asscat2_hdu = fits.PrimaryHDU(reproject.reproject_interp(asscat2_hdu, hst01_hdu.header, return_footprint=False, order='nearest-neighbor', parallel=True), hst01_hdu.header)
asscat3_hdu = fits.PrimaryHDU(reproject.reproject_interp(asscat3_hdu, hst01_hdu.header, return_footprint=False, order='nearest-neighbor', parallel=True), hst01_hdu.header)

asscat1_hdu.data[asscat1_hdu.data==0] = np.nan
asscat2_hdu.data[asscat2_hdu.data==0] = np.nan
asscat3_hdu.data[asscat3_hdu.data==0] = np.nan

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]


In [4]:
hdus = [hst01_hdu, hst02_hdu, hst03_hdu, hst04_hdu, hst05_hdu, hst06_hdu, hst07_hdu, musha_hdu,
        almaco_hdu, almaco1_hdu, vlahi_hdu, jwst01_hdu, jwst02_hdu, jwst03_hdu, jwst04_hdu,
        jwst05_hdu, jwst06_hdu, jwst07_hdu, jwst08_hdu, jwst09_hdu, jwst10_hdu, jwst11_hdu,
        jwst12_hdu, jwst13_hdu, jwst14_hdu, jwst15_hdu, jwst16_hdu, asto_hdu, spit_hdu,
        muscat_hdu, asscat1_hdu, asscat2_hdu, asscat3_hdu]

# Define the conversion function
def convert_to_float32(hdu):
    hdu.data = hdu.data.astype('float32')

# Create a ThreadPoolExecutor with the number of workers
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit conversion tasks to the executor
    futures = [executor.submit(convert_to_float32, hdu) for hdu in hdus]

    # Wait for all tasks to complete and track the progress
    with tqdm(total=len(futures), desc="Converting to float32") as pbar:
        for future in concurrent.futures.as_completed(futures):
            # Update the progress bar for each completed task
            pbar.update(1)

Converting to float32:   0%|          | 0/33 [00:00<?, ?it/s]

In [5]:
# Specify the regions file
regions_file = './../../analysis/catalogue/sample_v2.reg'

# Call the get_regions function to retrieve the region properties with a progress bar
regions = dendro_cutouts.get_regions(regions_file, hst01_hdu)

n = len(regions['ra'])
print('[INFO] Number of HII regions in sample --> %i' %n)

Processing:   0%|          | 0/215 [00:00<?, ?it/s]

[INFO] Number of HII regions in sample --> 215


In [6]:
# hdus = get_croppeddata_parallel(hst01_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst01_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(hst02_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst02_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(hst03_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst03_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(hst04_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst04_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(hst05_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst05_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(hst06_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst06_hdus.pickel') 
# del hdus
# _ = gc.collect()

hdus = dendro_cutouts.get_croppeddata_parallel(hst07_hdu, regions)
dendro_misc.save_pickle(hdus, '../../analysis/catalogue/hst07_hdus.pickel') 
del hdus
_ = gc.collect()

# hdus = get_croppeddata_parallel(musha_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/musha_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(almaco_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/almaco_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(almaco1_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/almaco1_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(vlahi_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/vlahi_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst01_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst01_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst02_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst02_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst03_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst03_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst04_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst04_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst05_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst05_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst06_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst06_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst07_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst07_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst08_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst08_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst09_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst09_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst10_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst10_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst11_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst11_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst12_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst12_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst13_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst13_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst14_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst14_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst15_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst15_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst16_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst16_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(muscat_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/muscat_hdus.pickel')
# del hdus

# hdus = get_croppeddata_parallel(asto_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/asto_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(spit_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/spit_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(asscat1_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/asscat1_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(asscat2_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/asscat2_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(asscat3_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/asscat3_hdus.pickel') 
# del hdus
# _ = gc.collect()

Cropping regions:   0%|          | 0/215 [00:00<?, ?it/s]

[INFO] Saved to ../../analysis/catalogue/hst07_hdus.pickel


In [7]:
hst01_hdus = dendro_misc.load_pickle('../../analysis/catalogue/hst01_hdus.pickel')
hst02_hdus = dendro_misc.load_pickle('../../analysis/catalogue/hst02_hdus.pickel')
hst03_hdus = dendro_misc.load_pickle('../../analysis/catalogue/hst03_hdus.pickel')
hst04_hdus = dendro_misc.load_pickle('../../analysis/catalogue/hst04_hdus.pickel')
hst05_hdus = dendro_misc.load_pickle('../../analysis/catalogue/hst05_hdus.pickel')
hst06_hdus = dendro_misc.load_pickle('../../analysis/catalogue/hst06_hdus.pickel')
hst07_hdus = dendro_misc.load_pickle('../../analysis/catalogue/hst07_hdus.pickel')
musha_hdus = dendro_misc.load_pickle('../../analysis/catalogue/musha_hdus.pickel')
almaco_hdus = dendro_misc.load_pickle('../../analysis/catalogue/almaco_hdus.pickel')
almaco1_hdus = dendro_misc.load_pickle('../../analysis/catalogue/almaco1_hdus.pickel')
vlahi_hdus = dendro_misc.load_pickle('../../analysis/catalogue/vlahi_hdus.pickel')
jwst01_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst01_hdus.pickel')
jwst02_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst02_hdus.pickel')
jwst03_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst03_hdus.pickel')
jwst04_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst04_hdus.pickel')
jwst05_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst05_hdus.pickel')
jwst06_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst06_hdus.pickel')
jwst07_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst07_hdus.pickel')
jwst08_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst08_hdus.pickel')
jwst09_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst09_hdus.pickel')
jwst10_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst10_hdus.pickel')
jwst11_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst11_hdus.pickel')
jwst12_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst12_hdus.pickel')
jwst13_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst13_hdus.pickel')
jwst14_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst14_hdus.pickel')
jwst15_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst15_hdus.pickel')
jwst16_hdus = dendro_misc.load_pickle('../../analysis/catalogue/jwst16_hdus.pickel')
asto_hdus = dendro_misc.load_pickle('../../analysis/catalogue/asto_hdus.pickel')
spit_hdus = dendro_misc.load_pickle('../../analysis/catalogue/spit_hdus.pickel')
muscat_hdus = dendro_misc.load_pickle('../../analysis/catalogue/muscat_hdus.pickel')
asscat1_hdus = dendro_misc.load_pickle('../../analysis/catalogue/asscat1_hdus.pickel')
asscat2_hdus = dendro_misc.load_pickle('../../analysis/catalogue/asscat2_hdus.pickel')
asscat3_hdus = dendro_misc.load_pickle('../../analysis/catalogue/asscat3_hdus.pickel')

hdus = {'hst01_hdus' : hst01_hdus,
        'hst02_hdus' : hst02_hdus,
        'hst03_hdus' : hst03_hdus,
        'hst04_hdus' : hst04_hdus,
        'hst05_hdus' : hst05_hdus, 
        'hst06_hdus' : hst06_hdus,
        'hst07_hdus' : hst07_hdus,
        'musha_hdus' : musha_hdus,
        'almaco_hdus' : almaco_hdus,
        'almaco1_hdus' : almaco1_hdus,
        'vlahi_hdus' : vlahi_hdus ,
        'jwst01_hdus' : jwst01_hdus, 
        'jwst02_hdus' : jwst02_hdus,
        'jwst03_hdus' : jwst03_hdus,
        'jwst04_hdus' : jwst04_hdus,
        'jwst05_hdus' : jwst05_hdus,
        'jwst06_hdus' : jwst06_hdus,
        'jwst07_hdus' : jwst07_hdus,
        'jwst08_hdus' : jwst08_hdus,
        'jwst09_hdus' : jwst09_hdus,
        'jwst10_hdus' : jwst10_hdus,
        'jwst11_hdus' : jwst11_hdus,
        'jwst12_hdus' : jwst12_hdus,
        'jwst13_hdus' : jwst13_hdus,
        'jwst14_hdus' : jwst14_hdus,
        'jwst15_hdus' : jwst15_hdus,
        'jwst16_hdus' : jwst16_hdus,
        'asto_hdus' : asto_hdus,
        'spit_hdus' : spit_hdus,
        'muscat_hdus' : muscat_hdus,
        'asscat1_hdus' : asscat1_hdus,
        'asscat2_hdus' : asscat2_hdus,
        'asscat3_hdus' : asscat3_hdus}

dendro_misc.save_pickle(hdus, '../../analysis/catalogue/hdus_sample_full_v3.pickel') 

[INFO] Loaded ../../analysis/catalogue/hst01_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/hst02_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/hst03_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/hst04_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/hst05_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/hst06_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/hst07_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/musha_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/almaco_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/almaco1_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/vlahi_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/jwst01_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/jwst02_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/jwst03_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/jwst04_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/jwst05_hdus.pickel
[INFO] Loaded ../../analysis/catalogue/jwst06_hdus.pickel
[INFO] Loaded ../../an

In [8]:
hdus = {'hst06_hdus' : hst06_hdus,
        'hst07_hdus' : hst07_hdus,
        'musha_hdus' : musha_hdus,
        'muscat_hdus' : muscat_hdus,
        'asscat1_hdus' : asscat1_hdus,
        'asscat2_hdus' : asscat2_hdus,
        'asscat3_hdus' : asscat3_hdus}

dendro_misc.save_pickle(hdus, '../../analysis/catalogue/hdus_sample_sci_v3.pickel') 

[INFO] Saved to ../../analysis/catalogue/hdus_sample_sci_v3.pickel
